In [136]:
import numpy as np
import pandas as pd
import datetime
import math
from os import listdir
from os.path import isfile, join
import json

In [137]:
out='../output/'
pref='dam'
inp='../input/monthly/' + pref + '/'


In [189]:
dat=pd.DataFrame(columns=['date','hour','price','volume'])
onlyfiles = [f for f in listdir(inp) if isfile(join(inp, f))]
for f in onlyfiles:
    print(f.split('_'))
    if f.split('_')[0]==pref:
        if f.split('.')[1]=='csv':
            data = pd.read_csv(join(inp, f))
            dat=pd.concat([dat,data])
            
dat['date']=pd.to_datetime(dat.date,format="%Y-%m-%d")

dat.sort_index(inplace=True)
dat       

['dam', '2016', '01.csv']
['dam', '2016', '01.json']
['dam', '2016', '02.csv']
['dam', '2016', '02.json']
['dam', '2016', '03.csv']
['dam', '2016', '03.json']
['dam', '2016', '04.csv']
['dam', '2016', '04.json']
['dam', '2016', '05.csv']
['dam', '2016', '05.json']
['dam', '2016', '06.csv']
['dam', '2016', '06.json']
['dam', '2016', '07.csv']
['dam', '2016', '07.json']
['dam', '2016', '08.csv']
['dam', '2016', '08.json']
['dam', '2016', '09.csv']
['dam', '2016', '09.json']
['dam', '2016', '10.csv']
['dam', '2016', '10.json']
['dam', '2016', '11.csv']
['dam', '2016', '11.json']
['dam', '2016', '12.csv']
['dam', '2016', '12.json']
['dam', '2017', '01.csv']
['dam', '2017', '01.json']
['dam', '2017', '02.csv']
['dam', '2017', '02.json']
['dam', '2017', '03.csv']
['dam', '2017', '03.json']
['dam', '2017', '04.csv']
['dam', '2017', '04.json']
['dam', '2017', '05.csv']
['dam', '2017', '05.json']
['dam', '2017', '06.csv']
['dam', '2017', '06.json']
['dam', '2017', '07.csv']
['dam', '2017', '07.

C:\Users\Andy Fleming\AppData\Local\Temp\ipykernel_25276\2425000150.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dat=pd.concat([dat,data])


,date,hour,price,volume
0,2022-12-01,0,226.10,27.0
0,2019-08-01,0,54.05,485.9
0,2018-11-01,0,29.98,463.0
0,2019-05-01,0,37.15,220.5
0,2024-05-01,0,25.03,58.0
...,...,...,...,...
1268,2024-01-31,19,199.94,68.0
1269,2024-01-31,20,200.25,80.0
1270,2024-01-31,21,199.07,100.0
1271,2024-01-31,22,158.00,101.0


In [190]:
hourly=dat.set_index(pd.DatetimeIndex(dat['date']))
hourly.sort_index(inplace=True)
hourly['year']=hourly.index.year
hourly['month']=hourly.index.month
hourly['day']=hourly.index.day
hourly['day_of_week']=hourly.index.dayofweek
hourly=hourly.drop(['date'],axis=1)
hourly['hour']=hourly['hour'].astype(int)
hourly

,hour,price,volume,year,month,day,day_of_week
date,,,,,,,
2016-01-01,1,40.08,486.6,2016,1,1,4
2016-01-01,7,48.03,350.0,2016,1,1,4
2016-01-01,8,45.10,323.0,2016,1,1,4
2016-01-01,13,45.10,323.0,2016,1,1,4
2016-01-01,3,40.08,496.6,2016,1,1,4
...,...,...,...,...,...,...,...
2024-06-30,14,109.96,33.0,2024,6,30,6
2024-06-30,9,120.32,102.0,2024,6,30,6
2024-06-30,13,109.95,28.0,2024,6,30,6


In [191]:
hourly['value']=hourly['price']*hourly['volume']
daily=hourly.groupby(['year','month','day']).sum().drop(['hour','day_of_week'],axis=1)
daily['price_weight_av']=daily['value']/daily['volume']
daily=daily.drop(['price'], axis=1)
daily['price_max']=hourly[['year','month','day','price']].groupby(['year','month','day']).max()
daily['price_min']=hourly[['year','month','day','price']].groupby(['year','month','day']).min()
daily['price_mean']=hourly[['year','month','day','price']].groupby(['year','month','day']).mean()

daily['power_max']=hourly[['year','month','day','volume']].groupby(['year','month','day']).max()
daily['power_min']=hourly[['year','month','day','volume']].groupby(['year','month','day']).min()
daily['power_mean']=hourly[['year','month','day','volume']].groupby(['year','month','day']).mean()

daily=daily.reset_index()
daily['date']=pd.to_datetime(daily.reset_index()[['year', 'month', 'day']])
daily=daily.set_index(['year','month','day'])
daily.sort_index(inplace=True)
daily


volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2024 6     26   1334.0  271031.120       203.171754     312.80     120.05   
           27   1337.0  282978.960       211.652177     306.54     120.04   
           28   1287.0  277650.800       215.734887     306.46     117.04   
           29   2198.0  363924.130       165.570578     308.35     116.90   
           30   2148.0  284966.260       132.665857     250.10     109.95   

                price_mean  power_max  power_min  power_mean       date  
year month day                                                           
2016 1     1     45.982500      502.9       53.0  328.075000 2016-01-01  
           2     54.114167      486.0       40.0  304.979167 2016-01-02  
           3     54.959167      486.0       40.0  299.945833 2016-01-03  
           4     80.164583      486.0        0.0  185.708333 2016-01-04  
           5     84.503333      436.0        0.0  156.266667 2016-01-05  
...                    ...        ...        ...         ...        ...  
2024 6     26   193.988750       93.0        8.0   55.583333 2024-06-26  
           27   204.397917       93.0       33.0   55.708333 2024-06-27  
           28   203.501250       93.0       33.0   53.625000 2024-06-28  
           29   159.089167      178.0       49.0   91.583333 2024-06-29  
           30   131.244583      151.0       28.0   89.500000 2024-06-30  

[3104 rows x 10 columns]

In [192]:
hourly['cat']='off'

if (pref=='dam' or pref=='idm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==6),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=7) & (hourly['hour']<=9),'cat']='morning'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=10) & (hourly['hour']<=16),'cat']='standard'

    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==17) & ((hourly['month']<=5) | (hourly['month']>=9)),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']<=3),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==19),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='evening'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==20) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==21),'cat']='standard'

    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==17) & (hourly['month']>=6) & (hourly['month']<=8),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==18) & (hourly['month']>=4),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==19),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']==20) & (hourly['month']<=3),'cat']='standard'

if (pref=='fpm'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=21),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'

if (pref=='fpw'):
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']==5),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=6) & (hourly['hour']<=8),'cat']='morning'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=9) & (hourly['hour']<=17),'cat']='standard'
    hourly.loc[(hourly['day_of_week']<5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='evening'
    
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=7) & (hourly['hour']<=11),'cat']='standard'
    hourly.loc[(hourly['day_of_week']==5) & (hourly['hour']>=18) & (hourly['hour']<=19),'cat']='standard'
    
hourly

,hour,price,volume,year,month,day,day_of_week,value,cat
date,,,,,,,,,
2016-01-01,1,40.08,486.6,2016,1,1,4,19502.928,off
2016-01-01,7,48.03,350.0,2016,1,1,4,16810.500,morning
2016-01-01,8,45.10,323.0,2016,1,1,4,14567.300,morning
2016-01-01,13,45.10,323.0,2016,1,1,4,14567.300,standard
2016-01-01,3,40.08,496.6,2016,1,1,4,19903.728,off
...,...,...,...,...,...,...,...,...,...
2024-06-30,14,109.96,33.0,2024,6,30,6,3628.680,off
2024-06-30,9,120.32,102.0,2024,6,30,6,12272.640,off
2024-06-30,13,109.95,28.0,2024,6,30,6,3078.600,off


In [196]:
daily['off_price_mean']=hourly.loc[hourly['cat']=='off'][['year','month','day','price']].groupby(['year','month','day']).mean()
daily['standard_price_mean']=hourly.loc[hourly['cat']=='standard'][['year','month','day','price']].groupby(['year','month','day']).mean()
daily['morning_price_mean']=hourly.loc[hourly['cat']=='morning'][['year','month','day','price']].groupby(['year','month','day']).mean()
daily['evening_price_mean']=hourly.loc[hourly['cat']=='evening'][['year','month','day','price']].groupby(['year','month','day']).mean()

daily['off_hour_count']=hourly.loc[hourly['cat']=='off'][['year','month','day','price']].groupby(['year','month','day']).count()
daily['standard_hour_count']=hourly.loc[hourly['cat']=='standard'][['year','month','day','price']].groupby(['year','month','day']).count()
daily['morning_hour_count']=hourly.loc[hourly['cat']=='morning'][['year','month','day','price']].groupby(['year','month','day']).count()
daily['evening_hour_count']=hourly.loc[hourly['cat']=='evening'][['year','month','day','price']].groupby(['year','month','day']).count()

daily['off_volume']=hourly.loc[hourly['cat']=='off'][['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['standard_volume']=hourly.loc[hourly['cat']=='standard'][['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['morning_volume']=hourly.loc[hourly['cat']=='morning'][['year','month','day','volume']].groupby(['year','month','day']).sum()
daily['evening_volume']=hourly.loc[hourly['cat']=='evening'][['year','month','day','volume']].groupby(['year','month','day']).sum()

daily['off_value']=hourly.loc[hourly['cat']=='off'][['year','month','day','value']].groupby(['year','month','day']).sum()
daily['standard_value']=hourly.loc[hourly['cat']=='standard'][['year','month','day','value']].groupby(['year','month','day']).sum()
daily['morning_value']=hourly.loc[hourly['cat']=='morning'][['year','month','day','value']].groupby(['year','month','day']).sum()
daily['evening_value']=hourly.loc[hourly['cat']=='evening'][['year','month','day','value']].groupby(['year','month','day']).sum()

daily

volume       value  price_weight_av  price_max  price_min  \
year month day                                                              
2016 1     1    7873.8  336882.508        42.785251      60.08      35.09   
           2    7319.5  345982.021        47.268532     118.49      40.03   
           3    7198.7  351712.733        48.857812     118.47      44.06   
           4    4457.0  225938.810        50.693024     177.66      46.70   
           5    3750.4  179068.882        47.746609     181.40      42.06   
...                ...         ...              ...        ...        ...   
2024 6     26   1334.0  271031.120       203.171754     312.80     120.05   
           27   1337.0  282978.960       211.652177     306.54     120.04   
           28   1287.0  277650.800       215.734887     306.46     117.04   
           29   2198.0  363924.130       165.570578     308.35     116.90   
           30   2148.0  284966.260       132.665857     250.10     109.95   

                price_mean  power_max  power_min  power_mean       date  ...  \
year month day                                                           ...   
2016 1     1     45.982500      502.9       53.0  328.075000 2016-01-01  ...   
           2     54.114167      486.0       40.0  304.979167 2016-01-02  ...   
           3     54.959167      486.0       40.0  299.945833 2016-01-03  ...   
           4     80.164583      486.0        0.0  185.708333 2016-01-04  ...   
           5     84.503333      436.0        0.0  156.266667 2016-01-05  ...   
...                    ...        ...        ...         ...        ...  ...   
2024 6     26   193.988750       93.0        8.0   55.583333 2024-06-26  ...   
           27   204.397917       93.0       33.0   55.708333 2024-06-27  ...   
           28   203.501250       93.0       33.0   53.625000 2024-06-28  ...   
           29   159.089167      178.0       49.0   91.583333 2024-06-29  ...   
           30   131.244583      151.0       28.0   89.500000 2024-06-30  ...   

                morning_hour_count  evening_hour_count  off_volume  \
year month day                                                       
2016 1     1                   3.0                 2.0      3851.5   
           2                   NaN                 NaN      5969.5   
           3                   NaN                 NaN      7198.7   
           4                   3.0                 2.0      3828.0   
           5                   3.0                 2.0      3230.0   
...                            ...                 ...         ...   
2024 6     26                  3.0                 3.0       455.0   
           27                  3.0                 3.0       426.0   
           28                  3.0                 3.0       350.0   
           29                  NaN                 NaN      1370.0   
           30                  NaN                 NaN      2148.0   

                standard_volume  morning_volume  evening_volume   off_value  \
year month day                                                                
2016 1     1             2920.3           993.0           109.0  152306.418   
           2             1350.0             NaN             NaN  271567.021   
           3                NaN             NaN             NaN  351712.733   
           4              464.0           109.0            56.0  179880.680   
           5              368.4           109.0            43.0  136366.420   
...                         ...             ...             ...         ...   
2024 6     26             501.0           194.0           184.0   54725.840   
           27             525.0           192.0           194.0   53621.920   
           28             534.0           209.0           194.0   44354.820   
           29             828.0             NaN             NaN  180133.510   
           30               NaN             NaN             NaN  284966.260   

                standard_value  mo

In [197]:
daily.to_csv(out + pref + '_daily.csv')

In [198]:
hourly.to_csv(out + pref + '_hourly.csv')

In [199]:
hourly_flat=hourly.reset_index().set_index(['date'])
hourly_slim=hourly_flat[['hour','price','volume','cat']]
hourly_slim

,hour,price,volume,cat
date,,,,
2016-01-01,1,40.08,486.6,off
2016-01-01,7,48.03,350.0,morning
2016-01-01,8,45.10,323.0,morning
2016-01-01,13,45.10,323.0,standard
2016-01-01,3,40.08,496.6,off
...,...,...,...,...
2024-06-30,14,109.96,33.0,off
2024-06-30,9,120.32,102.0,off
2024-06-30,13,109.95,28.0,off


In [200]:
hourly_slim.to_csv(out + pref + '_hourly_slim.csv')

In [201]:
monthly=daily.groupby(['year','month']).mean()
monthly=monthly.drop(columns=['morning_volume','off_volume','standard_volume','evening_volume','price_weight_av'], axis = 1)
monthly.rename(columns={'value':'value_mean','volume':'volume_mean','off_hour_count':'off_hour_mean','standard_hour_count':'standard_hour_mean','morning_hour_count':'morning_hour_mean','evening_hour_count':'evening_hour_mean'}, inplace=True)
monthly[['price_max','standard_price_max','off_price_max','morning_price_max','evening_price_max']]=daily.groupby(['year','month']).max()[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_min','standard_price_min','off_price_min','morning_price_min','evening_price_min']]=daily.groupby(['year','month']).min()[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]
monthly[['price_stdev','standard_price_stdev','off_price_stdev','morning_price_stdev','evening_price_stdev']]=daily.groupby(['year','month']).std()[['price_mean','standard_price_mean','off_price_mean','morning_price_mean','evening_price_mean']]

monthly

volume_mean     value_mean   price_max  price_min  price_mean  \
year month                                                                  
2016 1      3877.909677  216538.697903  104.036250  45.982500   81.784624   
     2      3915.110345  263339.799483  111.843750  75.023333   91.385273   
     3      3004.254839  192831.319548  101.464167  60.115833   88.099516   
     4      1934.450000  119384.364433  114.641667  40.925417   85.320472   
     5      1714.970968   95755.502839  104.820833  28.659583   78.638495   
...                 ...            ...         ...        ...         ...   
2024 2      4986.931034  401522.993448  139.473333  51.815833   98.470690   
     3      2599.645161  296836.615806  154.980833  90.540833  131.941586   
     4      2959.733333  234563.848333  119.876667  44.986667   85.931083   
     5      2514.645161  127451.500968  109.072917  23.725833   61.285833   
     6      2157.466667  325214.858000  204.397917  75.228750  155.957861   

             power_max  power_min  power_mean                date  \
year month                                                          
2016 1      362.651613  17.606452  161.579570 2016-01-16 00:00:00   
     2      367.837931  23.231034  163.129598 2016-02-15 00:00:00   
     3      285.867742  18.232258  125.177285 2016-03-16 00:00:00   
     4      183.970000   7.236667   80.602083 2016-04-15 12:00:00   
     5      165.619355   4.661290   71.457124 2016-05-16 00:00:00   
...                ...        ...         ...                 ...   
2024 2      323.620690  57.206897  207.788793 2024-02-15 00:00:00   
     3      191.258065  19.741935  108.318548 2024-03-16 00:00:00   
     4      196.633333  35.766667  123.322222 2024-04-15 12:00:00   
     5      193.354839  27.935484  104.776882 2024-05-16 00:00:00   
     6      164.633333  32.200000   89.894444 2024-06-15 12:00:00   

            off_price_mean  ...  evening_price_max  standard_price_min  \
year month                  ...                                          
2016 1           52.483265  ...         173.895000           47.883636   
     2           57.262857  ...         230.985000          105.227143   
     3           55.448623  ...         159.310000           79.008182   
     4           53.100462  ...         192.770000           48.655714   
     5           44.707955  ...         180.210000           38.725714   
...                    ...  ...                ...                 ...   
2024 2           72.624674  ...         202.020000           78.970000   
     3          119.986066  ...         197.080000           77.424286   
     4           81.491467  ...         175.060000           45.235455   
     5           44.166293  ...         198.180000           31.464545   
     6          116.695563  ...         305.293333          101.403750   

            off_price_min  morning_price_min  evening_price_min  price_stdev  \
year month                                                                     
2016 1           39.45250          47.046667          60.050000    16.069174   
     2           50.48000         113.423333         120.085000     9.513109   
     3           49.53125          98.910000          89.340000    10.907181   
     4           29.95625          45.033333         114.790000    17.997428   
     5           27.36750          42.956667         102.150000    21.811428   
...                   ...                ...                ...          ...   
2024 2           39.72000          69.936667         140.035000    20.592492   
     3           90.37500          84.956667         100.085000    19.172325   
     4           42.87750          46.373333          49.975000    19.384425   
     5           16.95625          26.553333          85.970000    21.851216   
     6           72.63125         115.013333         193.396667    34.591774   

            standard_price_stdev  off_price_stdev  morning_price_stdev  \
year month                        

In [203]:
if (pref=='dam'):
    monthly['evening_days']=daily.loc[daily['evening_hour_count']>0].groupby(['year','month']).count()['volume']
    monthly['morning_days']=daily.loc[daily['morning_hour_count']>0].groupby(['year','month']).count()['volume']
    monthly['days']=daily.groupby(['year','month']).count()['volume']
    monthly[['volume','morning_volume','evening_volume','standard_volume','off_volume']]=daily.drop(columns=['date'],axis=1).groupby(['year','month']).sum()[['volume','morning_volume','evening_volume','standard_volume','off_volume']]
    monthly[['value','morning_value','evening_value','standard_value','off_value']]=daily.drop(columns=['date'],axis=1).groupby(['year','month']).sum()[['value','morning_value','evening_value','standard_value','off_value']]
    monthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=daily.drop(columns=['date'],axis=1).groupby(['year','month']).sum()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]

    monthly['price_weight_mean']=monthly['value'] / monthly['volume']
    monthly['off_price_weight_mean']=monthly['off_value'] / monthly['off_volume']
    monthly['standard_price_weight_mean']=monthly['standard_value'] / monthly['standard_volume']
    monthly['morning_price_weight_mean']=monthly['morning_value'] / monthly['morning_volume']
    monthly['evening_price_weight_mean']=monthly['evening_value'] / monthly['evening_volume']

    monthly['off_power']=monthly['off_volume'] / monthly['off_hour_count']
    monthly['morning_power']=monthly['morning_volume'] / monthly['morning_hour_count']
    monthly['standard_power']=monthly['standard_volume'] / monthly['standard_hour_count']
    monthly['evening_power']=monthly['evening_volume'] / monthly['evening_hour_count']
monthly

volume_mean     value_mean   price_max  price_min  price_mean  \
year month                                                                  
2016 1      3877.909677  216538.697903  104.036250  45.982500   81.784624   
     2      3915.110345  263339.799483  111.843750  75.023333   91.385273   
     3      3004.254839  192831.319548  101.464167  60.115833   88.099516   
     4      1934.450000  119384.364433  114.641667  40.925417   85.320472   
     5      1714.970968   95755.502839  104.820833  28.659583   78.638495   
...                 ...            ...         ...        ...         ...   
2024 2      4986.931034  401522.993448  139.473333  51.815833   98.470690   
     3      2599.645161  296836.615806  154.980833  90.540833  131.941586   
     4      2959.733333  234563.848333  119.876667  44.986667   85.931083   
     5      2514.645161  127451.500968  109.072917  23.725833   61.285833   
     6      2157.466667  325214.858000  204.397917  75.228750  155.957861   

             power_max  power_min  power_mean                date  \
year month                                                          
2016 1      362.651613  17.606452  161.579570 2016-01-16 00:00:00   
     2      367.837931  23.231034  163.129598 2016-02-15 00:00:00   
     3      285.867742  18.232258  125.177285 2016-03-16 00:00:00   
     4      183.970000   7.236667   80.602083 2016-04-15 12:00:00   
     5      165.619355   4.661290   71.457124 2016-05-16 00:00:00   
...                ...        ...         ...                 ...   
2024 2      323.620690  57.206897  207.788793 2024-02-15 00:00:00   
     3      191.258065  19.741935  108.318548 2024-03-16 00:00:00   
     4      196.633333  35.766667  123.322222 2024-04-15 12:00:00   
     5      193.354839  27.935484  104.776882 2024-05-16 00:00:00   
     6      164.633333  32.200000   89.894444 2024-06-15 12:00:00   

            off_price_mean  ...  evening_hour_count  price_weight_mean  \
year month                  ...                                          
2016 1           52.483265  ...                42.0          55.839026   
     2           57.262857  ...                42.0          67.262421   
     3           55.448623  ...                46.0          64.186073   
     4           53.100462  ...                42.0          61.714888   
     5           44.707955  ...                44.0          55.835058   
...                    ...  ...                 ...                ...   
2024 2           72.624674  ...                42.0          80.515048   
     3          119.986066  ...                42.0         114.183513   
     4           81.491467  ...                44.0          79.251683   
     5           44.166293  ...                46.0          50.683692   
     6          116.695563  ...                60.0         150.739227   

            off_price_weight_mean  standard_price_weight_mean  \
year month                                                      
2016 1                  50.385088                   81.002858   
     2                  56.542839                  106.547628   
     3                  55.124423                   98.717668   
     4                  56.919792                  104.192200   
     5                  49.682582                  106.019186   
...                           ...                         ...   
2024 2                  62.673911                  105.162668   
     3                 108.834982                  119.579062   
     4                  75.424290                   85.515912   
     5                  41.742497                   56.410526   
     6                 117.945515                  172.790203   

            morning_price_weight_mean  evening_price_weight_mean   off_power  \
year month                                                                     
2016 1                      77.850006                 137.720998  272.092493   
     2                     129.026815                 156.80124

In [204]:
monthly['off_price_diff']=monthly['off_price_mean']-monthly['off_price_mean'].shift(1)
monthly['morning_price_diff']=monthly['morning_price_mean']-monthly['morning_price_mean'].shift(1)
monthly['standard_price_diff']=monthly['standard_price_mean']-monthly['standard_price_mean'].shift(1)
monthly['evening_price_diff']=monthly['evening_price_mean']-monthly['evening_price_mean'].shift(1)

#monthly['off_price_log_diff']=np.log10(monthly['off_price_mean'])-np.log10(monthly['off_price_mean']).shift(1)
#monthly['morning_price_log_diff']=np.log10(monthly['morning_price_mean'])-np.log10(monthly['morning_price_mean']).shift(1)
#monthly['standard_price_log_diff']=np.log10(monthly['standard_price_mean'])-np.log10(monthly['standard_price_mean']).shift(1)
#monthly['evening_price_log_diff']=np.log10(monthly['evening_price_mean'])-np.log10(monthly['evening_price_mean']).shift(1)

In [205]:
monthly.to_csv(out + pref + '_monthly.csv')

In [217]:
monthly_filt=monthly.reset_index().loc[monthly.reset_index()['year']>2019]
daily_filt=daily.reset_index().loc[daily.reset_index()['year']>2019]
hourly_filt=hourly.reset_index().loc[hourly.reset_index()['year']>2019]

In [222]:
daily_filt.groupby(['month']).mean().reset_index()

,month,year,day,volume,value,price_weight_av,price_max,price_min,price_mean,power_max,...,morning_hour_count,evening_hour_count,off_volume,standard_volume,morning_volume,evening_volume,off_value,standard_value,morning_value,evening_value
0,1,2022.0,16.000000,4056.654194,229530.505903,59.038354,194.851548,31.958774,68.255377,240.639355,...,3.436364,2.290909,2168.557419,1642.961364,546.358182,142.588182,85793.446994,112462.884886,41790.371055,25792.750091
1,2,2022.0,14.704225,2978.566197,221234.661711,76.215971,238.925704,41.306479,87.430472,203.315493,...,3.000000,2.000000,1514.814085,1250.210656,437.270297,110.522772,78332.750775,112390.278836,41516.572644,23636.470277
2,3,2022.0,16.000000,2437.424516,194256.565806,82.250711,205.520129,54.952581,93.369465,172.650323,...,3.000000,2.000000,1289.994194,1008.109774,287.894643,102.936607,81712.538994,89115.516346,30905.673902,19022.544687
3,4,2022.0,15.500000,2854.014000,164458.704287,61.801121,185.388800,39.026133,71.000436,191.176667,...,3.000000,2.000000,1540.588667,1125.703101,365.782243,118.312150,66335.352560,75940.762574,28024.603925,17976.745486
4,5,2022.0,16.000000,2856.050323,141099.367368,51.823293,178.675097,25.799484,59.179454,194.097419,...,3.000000,2.000000,1499.787097,1132.482576,410.083636,142.035455,47038.930239,66460.499159,28271.101600,24516.006273
5,6,2022.0,15.500000,2937.424667,211760.146320,82.235173,207.923667,41.723067,87.225397,207.105333,...,3.000000,3.000000,1482.616667,1120.238760,410.431481,272.072222,62173.755927,90641.271287,47678.466157,51814.446463
6,7,2021.5,16.000000,3573.600806,246897.843298,73.275103,211.395000,32.050887,79.566865,237.758871,...,3.000000,3.000000,1935.428226,1329.410377,411.354023,303.771264,63614.544871,108694.051170,58162.383460,70637.151954
7,8,2021.5,16.000000,3623.832258,257631.499548,73.693209,208.737258,30.609516,80.522157,237.330645,...,3.000000,3.000000,1889.662903,1398.433962,440.180899,310.414607,64357.091452,113551.363745,61381.551506,72658.696213
8,9,2021.5,15.500000,3611.050000,284379.131392,80.608367,213.147667,38.351167,92.365917,237.151667,...,3.000000,2.000000,1996.450000,1373.412500,419.678161,165.575862,93106.936008,139943.317144,61247.679862,35287.474885
9,10,2021.5,16.000000,3297.792742,323549.758960,100.578802,247.054113,48.950726,113.716193,222.979839,...,3.000000,2.000000,1862.229032,1248.623810,390.645349,154.754651,101588.509032,171115.505695,68504.193442,42612.863453


In [212]:
calmonthly=daily_filt.groupby(['month']).mean().reset_index()
calmonthly=calmonthly.drop(columns=['year','day','volume','standard_volume','off_volume','morning_volume','evening_volume', 'off_hour_count', 'morning_hour_count', 'standard_hour_count','evening_hour_count'], axis=1)
calmonthly.rename(columns={'value':'value_mean','volume':'volume_mean','price_weight_av': 'price_weight_mean'}, inplace=True)

calmonthly[['price_median', 'off_price_median','standard_price_median', 'morning_price_median', 'evening_price_median']]=daily_filt.groupby(['month']).median()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]
calmonthly[['price_stdev', 'off_price_stdev','standard_price_stdev', 'morning_price_stdev', 'evening_price_stdev']]=daily_filt.groupby(['month']).std()[['price_mean','off_price_mean','standard_price_mean', 'morning_price_mean', 'evening_price_mean']]

calmonthly[['volume', 'off_volume','standard_volume', 'morning_volume', 'evening_volume']]=monthly_filt.groupby(['month']).mean()[['volume','off_volume','standard_volume', 'morning_volume', 'evening_volume']]
calmonthly[['value', 'off_value','standard_value', 'morning_value', 'evening_value']]=monthly_filt.groupby(['month']).mean()[['value','off_value','standard_value', 'morning_value', 'evening_value']]
calmonthly[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]=monthly_filt.groupby(['month']).mean()[['off_hour_count','standard_hour_count', 'morning_hour_count', 'evening_hour_count']]
calmonthly[['off_power','standard_power', 'morning_power', 'evening_power']]=monthly_filt.groupby(['month']).mean()[['off_power','standard_power', 'morning_power', 'evening_power']]

calmonthly

,month,value_mean,price_weight_mean,price_max,price_min,price_mean,power_max,power_min,power_mean,date,...,evening_volume,value,off_hour_count,standard_hour_count,morning_hour_count,evening_hour_count,off_power,standard_power,morning_power,evening_power
0,1,229530.505903,59.038354,194.851548,31.958774,68.255377,240.639355,51.667742,147.203008,2022-01-15 19:12:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,221234.661711,76.215971,238.925704,41.306479,87.430472,203.315493,38.408451,124.106925,2022-02-14 12:00:00,...,3136.940,7.115446e+06,411.60,312.20,75.60,50.40,163.854105,137.289562,154.618321,61.422577
2,3,194256.565806,82.250711,205.520129,54.952581,93.369465,172.650323,36.896774,101.559355,2022-03-16 04:48:00,...,2232.560,6.283064e+06,329.00,251.60,60.60,40.40,130.925514,120.630754,144.930841,55.124238
3,4,164458.704287,61.801121,185.388800,39.026133,71.000436,191.176667,40.768000,118.917250,2022-04-15 16:48:00,...,2305.780,6.021954e+06,356.20,275.80,67.20,44.80,112.526145,97.243640,96.291971,50.949767
4,5,141099.367368,51.823293,178.675097,25.799484,59.179454,194.097419,51.968387,119.002097,2022-05-16 04:48:00,...,2531.880,4.933761e+06,346.80,266.20,64.20,42.80,133.658495,109.091127,122.333843,59.234571
5,6,211760.146320,82.235173,207.923667,41.723067,87.225397,207.105333,52.812000,122.392694,2022-06-15 16:48:00,...,3124.780,4.374080e+06,361.20,272.80,66.00,44.00,128.766234,109.425472,136.625857,70.752101
6,7,246897.843298,73.275103,211.395000,32.050887,79.566865,237.758871,65.497581,148.900034,2022-01-14 12:00:00,...,5876.760,6.352804e+06,340.80,249.60,64.80,64.80,130.999107,115.507897,135.967879,90.254848
7,8,257631.499548,73.693209,208.737258,30.609516,80.522157,237.330645,80.362097,150.993011,2022-02-14 12:00:00,...,6607.025,7.653833e+06,358.00,255.50,65.25,65.25,167.260663,138.229189,137.313963,101.944566
8,9,284379.131392,80.608367,213.147667,38.351167,92.365917,237.151667,61.968333,150.460417,2022-03-17 00:00:00,...,6906.725,7.986576e+06,354.00,256.50,66.75,66.75,165.673283,144.387539,146.946786,103.091189
9,10,323549.758960,100.578802,247.054113,48.950726,113.716193,222.979839,53.522581,137.408031,2022-04-16 12:00:00,...,3601.275,8.531374e+06,342.25,269.00,65.25,43.50,175.208966,132.659440,139.708947,82.268019


In [214]:
calmonthly.to_csv(out + pref + '_calmonthly.csv')

In [218]:
calmonthly_hours=hourly_filt.drop(columns=['date','cat'], axis=1).groupby(['month','hour']).mean()
calmonthly_hours=calmonthly_hours.drop(columns=['year','day','day_of_week'],axis=1)
calmonthly_hours

price      volume         value
month hour                                      
1     0      38.541017  169.483051   6166.284593
      1      36.164124  180.522034   6187.199102
      2      34.995028  187.305650   6236.732107
      3      34.566610  188.314689   6204.837881
      4      35.744859  188.744068   6393.709113
...                ...         ...           ...
12    19    203.903790   62.265323  10939.712613
      20    180.999032   56.056452   8617.212403
      21    143.345323   83.110484   9876.305177
      22     83.154677  114.162903   8396.729363
      23     65.472823  133.426613   7660.687540

[288 rows x 3 columns]

In [219]:
calmonthly_hours.to_csv(out + pref + '_calmonthly_hours.csv')

In [220]:
yearly_calmonthly_hours=hourly_filt.drop(columns=['date','cat'], axis=1).groupby(['year','month','hour']).mean()
yearly_calmonthly_hours=yearly_calmonthly_hours.drop(columns=['day','day_of_week'], axis=1)
yearly_calmonthly_hours

price      volume         value
year month hour                                      
2020 1     0      46.363548  156.645161   7225.481613
           1      43.721935  166.580645   7316.026129
           2      42.424516  170.161290   7284.921613
           3      42.173226  169.838710   7233.679032
           4      42.779355  176.035484   7619.790323
...                     ...         ...           ...
2024 6     19    249.598000   77.266667  18252.560667
           20    200.116000  101.866667  19494.609667
           21    164.225667   84.833333  13536.304333
           22    119.260667   91.466667  10467.632333
           23    109.977000   91.333333   9721.441000

[1296 rows x 3 columns]

In [221]:
yearly_calmonthly_hours.to_csv(out + pref + '_yearly_calmonthly_hours.csv')

In [143]:
from statsmodels.tsa.stattools import pacf

In [144]:
pacfs=pd.DataFrame(data={'standard': pacf(monthly.reset_index()['standard_price_mean'], nlags=14)})
pacfs['evening']= pacf(monthly.reset_index()['evening_price_mean'], nlags=14)
pacfs['morning']= pacf(monthly.reset_index()['morning_price_mean'], nlags=14)
pacfs['off']= pacf(monthly.reset_index()['off_price_mean'], nlags=14)
pacfs

,standard,evening,morning,off
0,1.000000,1.0,1.0,1.000000
1,0.575693,NaN,NaN,0.719482
2,-0.141688,NaN,NaN,0.156318
3,0.094764,NaN,NaN,0.076828
4,0.372824,NaN,NaN,0.146447
5,0.214987,NaN,NaN,-0.107136
6,0.050937,NaN,NaN,-0.169003
7,0.017093,NaN,NaN,0.142308
8,0.053606,NaN,NaN,-0.010914
9,-0.030040,NaN,NaN,0.085581


In [145]:
pacfs.to_csv(out + pref + '_pacfs.csv')